In [38]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision import datasets

from resnet import ResNet20

import os
import torch
import torch.optim as optim
import torch.nn as nn
from datetime import datetime

from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [39]:
hyperparameters = {
    'sparsity_type': "nm", # nm / entropy / feather / spartan / ses / base (no sparsity)
    'epochs': 200,
    'lr': 0.1,
    'momentum': 0.9,
    'weight_decay': 5e-4,
    'batch_size': 128,
}

In [40]:
transform_train = transforms.Compose([
    transforms.RandomCrop(size=32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                         std=[0.2023, 0.1994, 0.201]),
])

transform_val = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                         std=[0.2023, 0.1994, 0.201]),
])


train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_val)

train_loader = DataLoader(train_dataset, batch_size=hyperparameters['batch_size'], shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=hyperparameters['batch_size'], shuffle=False, num_workers=2)

In [41]:
class Pruner:
    def __init__(self, model, sparsity_ratio=0.5):
        self.sparsity_ratio = 0.5
        self.model = model

    def apply_nm_sparsity(self):
        """
        Apply N:M sparsity (2:4 pattern) to the ResNet-20 model.
        This applies after training is completed and should only affect inference time.
        """
        for name, module in self.model.named_modules():
            if isinstance(module, nn.Conv2d):
                # For each convolutional layer, apply 2:4 sparsity
                weight = module.weight.data
                num_filters = weight.size(0)  # Number of filters in the convolution
                
                # For every 4 weights, we keep the 2 largest magnitude weights and zero out the others
                for i in range(num_filters):
                    # Get the weights of the i-th filter
                    filter_weights = weight[i].view(-1)  # Flatten the filter weights
                    topk_values, topk_indices = torch.topk(torch.abs(filter_weights),
                                                           int(len(filter_weights) * self.sparsity_ratio))
                    
                    # Set the smallest 2 values in each 4-group to zero
                    threshold = topk_values[-1]  # The smallest non-zero magnitude in the top-k
                    weight[i].data[torch.abs(weight[i]) < threshold] = 0  # Prune the less important weights
                
                print(f"Applied N:M sparsity (2:4) on {name} filters")

    def print_sparsity(self):
        tot_params = 0
        tot_zeros = 0

        for name, module in self.model.named_modules():
            if isinstance(module, nn.Conv2d):
                weight = module.weight.data
                n_params = weight.numel()
                n_zeros = torch.sum(weight == 0).item()

                tot_params += n_params
                tot_zeros += n_zeros

                print(f"{name}: Total Params = {n_params}. Total zero params = {n_zeros}. Sparsity = {n_zeros / n_params:.2%}")
            

In [42]:
def train(model, train_loader, criterion, optimizer, epoch, log_file):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}", ncols=100)
    
    for batch_idx, (inputs, targets) in enumerate(pbar):
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, targets)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        correct += predicted.eq(targets).sum().item()
        total += targets.size(0)

        pbar.set_postfix(loss=running_loss/(batch_idx+1), accuracy=100.0 * correct / total)

    avg_loss = running_loss / len(train_loader)
    accuracy = 100.0 * correct / total
    log_file.write(f'Epoch [{epoch+1}], Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%\n')

    return avg_loss, accuracy


In [43]:
def test(model, test_loader, criterion, log_file):
    model.eval()
    correct = 0
    total = 0
    test_loss = 0.0

    pbar = tqdm(test_loader, desc="Testing", ncols=100)
    
    with torch.no_grad():
        for inputs, targets in pbar:
            inputs, targets = inputs.to(device), targets.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()

            _, predicted = outputs.max(1)
            correct += predicted.eq(targets).sum().item()
            total += targets.size(0)

            pbar.set_postfix(loss=test_loss/(total + inputs.size(0)), accuracy=100.0 * correct / total)

    avg_test_loss = test_loss / len(test_loader)
    accuracy = 100.0 * correct / total
    log_file.write(f'Test Loss: {avg_test_loss:.4f}, Accuracy: {accuracy:.2f}%\n')

    return avg_test_loss, accuracy

In [44]:
resnet20_model = ResNet20()
resnet20_model.to(device)

ResNet20(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bnorm1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (residual_layers): Sequential(
    (0): ResBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bnorm1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bnorm2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (residual): Sequential()
    )
    (1): ResBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bnorm1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bnorm2): BatchNorm2d(16, eps=1e-05,

In [45]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet20_model.parameters(), lr=hyperparameters['lr'], 
                      momentum=hyperparameters['momentum'], weight_decay=hyperparameters['weight_decay'])

In [46]:
current_learning_rate = hyperparameters['lr']
DECAY = 0.1
DECAY_EPOCHS = [100, 150]
EPOCHS = 200

In [47]:
sparsity_type = hyperparameters['sparsity_type']
output_base_path = "/home/sg666/Class/ECE661/outputs"
sparsity_folder_path = os.path.join(output_base_path, sparsity_type)

hyperparameter_str = f"epochs_{hyperparameters['epochs']}_lr_{hyperparameters['lr']}_momentum_{hyperparameters['momentum']}_wd_{hyperparameters['weight_decay']}_batch_{hyperparameters['batch_size']}"
output_folder = os.path.join(sparsity_folder_path, hyperparameter_str)

os.makedirs(output_folder, exist_ok=True)

hyperparameter_file = os.path.join(output_folder, 'hyperparameters.txt')
with open(hyperparameter_file, 'w') as f:
    for key, value in hyperparameters.items():
        f.write(f"{key}: {value}\n")

log_file_path = os.path.join(output_folder, 'training_log.txt')

In [48]:
pruner = Pruner(resnet20_model, sparsity_ratio=0.5)

In [49]:
# Apply N:M sparsity on the trained base ResNet-20, then train model again with sparsity
resnet20_model.load_state_dict(torch.load("/home/sg666/Class/ECE661/outputs/base/epochs_200_lr_0.1_momentum_0.9_wd_0.0005_batch_128/best_model.pth"))
pruner.apply_nm_sparsity()
pruner.print_sparsity()

Applied N:M sparsity (2:4) on conv1 filters
Applied N:M sparsity (2:4) on residual_layers.0.conv1 filters
Applied N:M sparsity (2:4) on residual_layers.0.conv2 filters
Applied N:M sparsity (2:4) on residual_layers.1.conv1 filters
Applied N:M sparsity (2:4) on residual_layers.1.conv2 filters
Applied N:M sparsity (2:4) on residual_layers.2.conv1 filters
Applied N:M sparsity (2:4) on residual_layers.2.conv2 filters
Applied N:M sparsity (2:4) on residual_layers.3.conv1 filters
Applied N:M sparsity (2:4) on residual_layers.3.conv2 filters
Applied N:M sparsity (2:4) on residual_layers.3.residual.0 filters
Applied N:M sparsity (2:4) on residual_layers.4.conv1 filters
Applied N:M sparsity (2:4) on residual_layers.4.conv2 filters
Applied N:M sparsity (2:4) on residual_layers.5.conv1 filters
Applied N:M sparsity (2:4) on residual_layers.5.conv2 filters
Applied N:M sparsity (2:4) on residual_layers.6.conv1 filters
Applied N:M sparsity (2:4) on residual_layers.6.conv2 filters
Applied N:M sparsity 

In [50]:
with open(log_file_path, 'w') as log_file:
    log_file.write(f"Training started at {datetime.now()}\n")

    best_accuracy = 0.0
    
    for epoch in range(hyperparameters['epochs']):
        train_loss, train_accuracy = train(resnet20_model, train_loader, criterion, optimizer, epoch, log_file)
        test_loss, test_accuracy = test(resnet20_model, test_loader, criterion, log_file)

        if test_accuracy > best_accuracy:
            best_accuracy = test_accuracy
            model_checkpoint_path = os.path.join(output_folder, f"model_best.pth")
            torch.save(resnet20_model.state_dict(), model_checkpoint_path)
            print(f"Saved best model at epoch {epoch+1} with accuracy: {best_accuracy:.2f}%")


        if epoch+1 in DECAY_EPOCHS:
            current_learning_rate = current_learning_rate * DECAY
            for param_group in optimizer.param_groups:
                param_group['lr'] = current_learning_rate
            print("Current learning rate has decayed to %f" %current_learning_rate)

    log_file.write(f"Training completed at {datetime.now()}\n")
    log_file.write(f"Best model accuracy: {best_accuracy:.2f}%\n")


Testing: 100%|██████████████████████████| 79/79 [00:00<00:00, 79.95it/s, accuracy=61.6, loss=0.0103]


Saved best model at epoch 1 with accuracy: 61.60%


Testing: 100%|█████████████████████████| 79/79 [00:01<00:00, 71.45it/s, accuracy=74.7, loss=0.00606]


Saved best model at epoch 2 with accuracy: 74.71%


Testing: 100%|█████████████████████████| 79/79 [00:01<00:00, 69.12it/s, accuracy=76.4, loss=0.00597]


...

Testing: 100%|█████████████████████████| 79/79 [00:00<00:00, 83.44it/s, accuracy=92.5, loss=0.00212]


Saved best model at epoch 174 with accuracy: 92.48%


Testing: 100%|██████████████████████████| 79/79 [00:01<00:00, 69.45it/s, accuracy=92.5, loss=0.0022]


Saved best model at epoch 185 with accuracy: 92.50%


Testing: 100%|█████████████████████████| 79/79 [00:01<00:00, 63.00it/s, accuracy=92.3, loss=0.00226]
